In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sqlalchemy import create_engine

In [3]:
os.chdir('./')

# Review nutrient table and get only important values for us

In [26]:
nut = pd.read_csv("./Health_Nutritions _databases/nutrient.csv", usecols=['id', 'name', 'unit_name'])
list(nut["name"])

['Nitrogen',
 'Protein',
 'Total lipid (fat)',
 'Carbohydrate, by difference',
 'Ash',
 'Energy',
 'Starch',
 'Sucrose',
 'Glucose (dextrose)',
 'Fructose',
 'Lactose',
 'Maltose',
 'Alcohol, ethyl',
 'Specific Gravity',
 'Acetic acid',
 'Citric acid',
 'Lactic acid',
 'Malic acid',
 'Carbohydrate, by summation',
 'Water',
 'Sorbitol',
 'Caffeine',
 'Theobromine',
 'Energy',
 'Sugars, Total NLEA',
 'Carbohydrate, other',
 'Galactose',
 'Xylitol',
 'Fiber, total dietary',
 'Ribose',
 'Fiber, soluble',
 'Fiber, insoluble',
 'Total fat (NLEA)',
 'Total sugar alcohols',
 'Calcium, Ca',
 'Chlorine, Cl',
 'Iron, Fe',
 'Magnesium, Mg',
 'Phosphorus, P',
 'Potassium, K',
 'Sodium, Na',
 'Sulfur, S',
 'Zinc, Zn',
 'Chromium, Cr',
 'Cobalt, Co',
 'Copper, Cu',
 'Fluoride, F',
 'Iodine, I',
 'Manganese, Mn',
 'Molybdenum, Mo',
 'Selenium, Se',
 'Vitamin A, IU',
 'Retinol',
 'Vitamin A, RAE',
 'Carotene, beta',
 'Carotene, alpha',
 'Vitamin E (alpha-tocopherol)',
 'Vitamin D (D2 + D3), Internation

In [28]:
nut_select = ['Protein', 'Energy', 'Carbohydrate, by difference', 'Sugars, Total NLEA', 'Fatty acids, total saturated', 'Fatty acids, total monounsaturated', 'Fatty acids, total polyunsaturated']
importnut = nut[nut['name'].isin(nut_select)]
importnut

,id,name,unit_name
1,1003,Protein,G
3,1005,"Carbohydrate, by difference",G
5,1008,Energy,KCAL
23,1062,Energy,kJ
24,1063,"Sugars, Total NLEA",G
136,1258,"Fatty acids, total saturated",G
166,1292,"Fatty acids, total monounsaturated",G
167,1293,"Fatty acids, total polyunsaturated",G


In [29]:
nut_list = list(importnut['id'])
nut_list #prepared list used later for narrowing spectrum 

[1003, 1005, 1008, 1062, 1063, 1258, 1292, 1293]

# Review food nutrient table

In [30]:
food_nut = pd.read_csv("./Health_Nutritions _databases/food_nutrient.csv", low_memory=False,
                      usecols=['fdc_id', 'nutrient_id','amount'])

In [31]:
food_nut = food_nut[food_nut['nutrient_id'].isin(nut_list)]

In [32]:
food_nut.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2059626 entries, 203 to 7547843
Data columns (total 3 columns):
 #   Column       Dtype  
---  ------       -----  
 0   fdc_id       int64  
 1   nutrient_id  int64  
 2   amount       float64
dtypes: float64(1), int64(2)
memory usage: 62.9 MB


# Review food table

In [33]:
data = pd.read_csv("./Health_Nutritions _databases/food.csv", low_memory=False,
                  usecols=['fdc_id', 'data_type', 'description'], index_col='fdc_id')
data

,data_type,description
fdc_id,,
1105898,experimental_food,Metabolizable Energy of Almonds
1105899,experimental_food,"Metabolizable Energy of Almonds, Food Processi..."
1105900,experimental_food,Metabolizable Energy of Cashews
1105901,experimental_food,Metabolizable Energy of Pistachios
1105902,experimental_food,Metabolizable Energy of Walnuts
...,...,...
1105893,sub_sample_food,"APPLES, FUJI"
1105894,sub_sample_food,"APPLES, FUJI"
1105895,sub_sample_food,"APPLES, FUJI"


In [43]:
search_word = input("Insert searching product: ")
search_list = data.where(data['description'].str.contains(search_word)).dropna(how='all')
search_list

Insert searching product: Rice


,data_type,description
fdc_id,,
168065,sr_legacy_food,"Restaurant, Latino, Arroz con habichuelas colo..."
168107,sr_legacy_food,"Rice cake, cracker (include hain mini rice cakes)"
168875,sr_legacy_food,"Rice, brown, medium-grain, cooked (Includes fo..."
168876,sr_legacy_food,"Rice, brown, parboiled, dry, UNCLE BEN'S"
168877,sr_legacy_food,"Rice, white, long-grain, regular, raw, enriched"
...,...,...
1102514,survey_fndds_food,"Rice, brown, with vegetables, soy-based sauce,..."
1102515,survey_fndds_food,"Rice, brown, with vegetables, soy-based sauce,..."
1102516,survey_fndds_food,"Rice, brown, with vegetables, soy-based sauce,..."


# Return nutrient values for choose product (fdc_id)

In [46]:
fdc = int(input("Insert fdc_id of product: "))
work = food_nut.where(food_nut['fdc_id'] == fdc).dropna().rename(columns = {"nutrient_id": "id"})
work

Insert fdc_id of product: 168877


,fdc_id,id,amount
112225,168877.0,1005.0,79.950
112226,168877.0,1008.0,365.000
112231,168877.0,1292.0,0.206
112232,168877.0,1293.0,0.177
112250,168877.0,1003.0,7.130
112264,168877.0,1258.0,0.180
112284,168877.0,1062.0,1527.000


#Display in format amount - nutrient - unit name`

In [47]:
result = pd.merge(importnut, work)[["amount", "name", "unit_name"]]
result

,amount,name,unit_name
0,7.130,Protein,G
1,79.950,"Carbohydrate, by difference",G
2,365.000,Energy,KCAL
3,1527.000,Energy,kJ
4,0.180,"Fatty acids, total saturated",G
5,0.206,"Fatty acids, total monounsaturated",G
6,0.177,"Fatty acids, total polyunsaturated",G
